# FINAL PROJECT

## 1. Collecting data

• What subject is your data about? What is the source of
your data?

• Do authors of this data allow you to use like this? You
can check the data license

• How did authors collect data?

## 2. Exploring data 
### (often interleaved with preprocessing)

• How many rows and how many columns?

• What is the meaning of each row?

• Are there `duplicated rows`?

• What is the meaning of each column?


• What is the current data type of each column? Are
there columns having `inappropriate data types`?

• With each numerical column, how are values
distributed?
    - What is the percentage of `missing values`?
    - Min? max? Are they `abnormal`?

• With each categorical column, how are values
distributed?
    - What is the percentage of `missing values`?
    - How many different values? Show a few
      Are they `abnormal`?

## 3. Asking meaningful questions

Your group needs to give ≥ `the-number-of-group-members`
questions which can be answered with this data. Each
question should be `meaningful` (what are benefits of finding
the answer?) and `not too easy to answer` (e.g., it’s too easy if
we just need one line of code to get the answer). Your
group should focus more on `the quality of questions` than
the quantity.

In notebook file, with each question, your group needs to
present:

• What is the question?

• What are benefits of finding the answer?

## 4. Preprocessing + analyzing data to answer each question

With each question:
• Does it need to have preprocessing step, and if yes,
how does your group preprocess?

• Text: sketch steps `clearly` so that readers can
understand how your group preprocesses even without
reading code

• Code: implement sketched steps. Your group should
also try to write code `clearly` (choose good variable
names, comment where should be commented, don’t
let a line too long)

• How does your group analyze data to answer the
question?

• Text: similar to above

• Code: similar to above

## 5. Reflection

• Each member: What difficulties have you encountered?

• Each member: What have you learned?

• Your group: If you had more time, what would you do?

## 6. References
To finish this project, what materials have you consulted?